In [27]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import cv2
import os
import time
import pysuperansac
import sys
import cv2
import io
import requests

In [28]:
# Loading example data
correspondences = np.loadtxt('data/rigid_pose_example_points.txt')
gt_pose = np.loadtxt('data/rigid_pose_example_gt.txt')
ground_truth_T = gt_pose[:4, :]

# Translating the points so there are no negative coordinates.
# This is only important if the space partitioning technique is used to
# accelerate the robust estimation, or when the spatial coherence term is >0.
min_coordinates = np.min(correspondences, axis=0)
T1 = np.array([[1,0,0,0], [0,1,0,0], [0,0,1,0], [-min_coordinates[0], -min_coordinates[1], -min_coordinates[2], 1]])
T2inv = np.array([[1,0,0,0], [0,1,0,0], [0,0,1,0], [min_coordinates[3], min_coordinates[4], min_coordinates[5], 1]])
transformed_correspondences = correspondences - min_coordinates

In [29]:
def run_superansac(matches, config):
    # Run the fundamental matrix estimation implemented in SupeRANSAC
    tic = time.perf_counter()
    T, inliers, score, iterations = pysuperansac.estimateRigidTransform(
        np.ascontiguousarray(matches), 
        np.max(matches, axis=0),
        config = config)
    toc = time.perf_counter()
    elapsed_time = toc - tic
    print (f'{len(inliers)} inliers found by SupeRANSAC in {elapsed_time:0.3f} seconds')

    mask = np.zeros((matches.shape[0], 1), dtype=np.uint8)
    mask[inliers] = 1

    return T, mask

def tranform_points(corrs, T):
    n = len(corrs)
    points1 = np.float32([corrs[i][0:3] for i in np.arange(n)]).reshape(-1,3)
    points2 = np.float32([corrs[i][3:6] for i in np.arange(n)]).reshape(-1,3)
    
    transformed_corrs = np.zeros((corrs.shape[0], 6))

    for i in range(n):
        p1 = np.append(correspondences[i][:3], 1)
        p2 = p1.dot(T)
        transformed_corrs[i][:3] = p2[:3]
        transformed_corrs[i][3:] = corrs[i][3:]
    return transformed_corrs
    

def calculate_error(gt_pose, est_pose):
    R2R1 = np.dot(gt_pose[:3, :3].T, est_pose[:3, :3])
    cos_angle = max(-1.0, min(1.0, 0.5 * (R2R1.trace() - 1.0)))
    
    err_R = np.arccos(cos_angle) * 180.0 / np.pi
    err_t = np.linalg.norm(gt_pose[:3, 3] - est_pose[:3, 3])
    
    return err_R, err_t

In [ ]:
# Set up the configuration
config = pysuperansac.RANSACSettings()
config.inlier_threshold = 0.2
config.min_iterations = 1000
config.max_iterations = 1000
config.confidence = 0.999
config.sampler = pysuperansac.SamplerType.PROSAC
config.scoring = pysuperansac.ScoringType.MAGSAC
config.local_optimization = pysuperansac.LocalOptimizationType.NestedRANSAC
config.final_optimization = pysuperansac.LocalOptimizationType.LSQ

# Run OpenCV RANSAC 
T_sr, mask_sr = run_superansac(transformed_correspondences, config)

if T_sr is None:
    T_sr = np.eye(4)
else:
    T_sr = T1 @ T_sr @ T2inv
    T_sr = T_sr.T
    
err_R, err_t = calculate_error(ground_truth_T, T_sr)

print ('Rotation error = ', err_R, '°')
print ('Translation error = ', err_t, ' cm')

3859 inliers found by SupeRANSAC in 0.201 seconds
Rotation error =  1.858955597097899 °
Translation error =  0.08333994088386605  cm
